In [45]:
import json
import string

import nltk
import requests
import operator
import numpy as np
import numpy.linalg as LA

import os
import pprint
import twitter

import newspaper
import requests
from newspaper import Article
from nytimesarticle import articleAPI

from twitter import read_token_file, oauth_dance, write_token_file

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

from miners.NewsMiner import NewsMiner
from miners.TwitterMiner import TwitterMiner

from textblob import TextBlob
from bs4 import BeautifulSoup

class TwitterMiner:
    def __init__(self):
        ""

    def login(self):
        APP_NAME = 'PPE'
        CONSUMER_KEY = 'JK9vbq72nrL8BwBBUhOKASosL'
        CONSUMER_SECRET = 'tQvz6zTa3nMbLMISPRMjFW6UDIpUBoDjMoFqg3zCI9SuoQrEHT'
        TOKEN_FILE = 'out/twitter.oauth'

        try:
            (oauth_token, oauth_token_secret) = read_token_file(TOKEN_FILE)
        except IOError as e:
            (oauth_token, oauth_token_secret) = oauth_dance(APP_NAME, CONSUMER_KEY,
                                                            CONSUMER_SECRET)

            if not os.path.isdir('out'):
                os.mkdir('out')

            write_token_file(TOKEN_FILE, oauth_token, oauth_token_secret)

        return twitter.Twitter(domain='api.twitter.com', api_version='1.1',
                               auth=twitter.oauth.OAuth(oauth_token, oauth_token_secret,
                                                        CONSUMER_KEY, CONSUMER_SECRET))

def get_tweets():
    return TwitterMiner().login().search.tweets(q="#trump", count=100)


def get_articles():
    nm = NewsMiner()
    articles = nm.get_articles("Donald Trump", "20170120", "20170830")
    nm.get_text(articles)
    with open('out/nyt_articles.json', 'w') as fout:
        json.dump(articles, fout)
    print(articles)

    # cnn = NewsMiner().sources['cnn']
    # for i in range(10):
    #     print (cnn.size())

    
class NewsMiner:
    def __init__(self):
        self.client = newspaper
        self.sources = {}
        self.article_api = articleAPI('60425d8974b1484692c368f8c52e4c1f')
        self.nyi_api_key = "60425d8974b1484692c368f8c52e4c1f"

    def setup(self):
        self.get_articles()
        # self.add_source()

    def add_source(self):
        self.sources['cnn'] = self.client.build('http://cnn.com', memoize_articles=False)

    def get_articles(self, search_term, begin_date, end_date):
        articles = []
        url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
        for page in range(100):
            queries = {
                'api-key': self.nyi_api_key,
                'q': search_term,
                'begin_date': begin_date,
                'end_date': end_date,
                'page': page
            }
            req_t = requests.get(url, params=queries)
            data = json.loads(req_t.text)
            if 'response' in data:
                articles.extend(data['response']['docs'])
        return articles

    def get_text(self, articles):
        for a in articles:
            try:
                article = Article(a['web_url'])
                article.download()
                article.parse()
                article.nlp()
                a['summary'] = article.summary
                a['keywords_1'] = article.keywords
                a['text'] = article.text
            except:
                pass
    

def get_tokens():
    corpus = ""
    with open("out/nyt_articles.json") as data_file:
        data = json.load(data_file)
    for article in data:
        corpus += article['text']
    lowers = corpus.lower()
    no_punctuation = lowers.translate(str.maketrans("", "", string.punctuation))
    toker = nltk.RegexpTokenizer(r'\w+')
    tokens = toker.tokenize(no_punctuation)
    count = nltk.Counter(tokens)
    print(count.most_common(10))
    return tokens


def remove_stop_words(tokens):
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    count = nltk.Counter(filtered)
    print(count.most_common(10))
    return filtered


def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        # stemmed.append(nltk.PorterStemmer().stem(item))
        stemmed.append(nltk.WordNetLemmatizer().lemmatize(item))
    count = nltk.Counter(stemmed)
    # print(count.most_common(10))
    return stemmed


def get_article_token():
    article_token_dict = {}
    toker = nltk.RegexpTokenizer(r'\w+')
    with open("out/nyt_articles.json") as data_file:
        data = json.load(data_file)
    for article in data:
        text = article['text']
        lowers = text.lower()
        no_punctuation = lowers.translate(str.maketrans("", "", string.punctuation))
        tokens = toker.tokenize(no_punctuation)
        article_token_dict[article['_id']] = " ".join(tokens)
    return article_token_dict


def tokenize(text):
    toker = nltk.RegexpTokenizer(r'\w+')
    lowers = text.lower()
    no_punctuation = lowers.translate(str.maketrans("", "", string.punctuation))
    tokens = toker.tokenize(no_punctuation)
    stems = stem_tokens(tokens)
    return stems


def get_promise_token():
    promise_token_dict = {}
    token = nltk.RegexpTokenizer(r'\w+')
    with open("out/promises2.json") as data_file:
        data = json.load(data_file)
    for promise in data:
        if promise['promise_description']:
            text = promise['promise_description']
        else:
            text = promise['promise_title']
        lowers = text.lower()
        no_punctuation = lowers.translate(str.maketrans("", "", string.punctuation))
        tokens = token.tokenize(no_punctuation)
        promise_token_dict[promise['promise_title']] = " ".join(tokens)
    return promise_token_dict


def google_search(search_query):
    search_sentiment_result = []
    search_query = search_query.replace(" ", "+")
    query = "https://www.google.com/search?q=" + search_query
    r = requests.get(query)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    for s in soup.find_all(attrs={'class': 'st'}):
        search_sentiment_result.append(sentiment_analysis(s.text))

    my_list = {i: search_sentiment_result.count(i) for i in search_sentiment_result}
    print(max(my_list.items(), key=operator.itemgetter(1))[0])


def sentiment_analysis(c):
    blob = TextBlob(c)
    for sentence in blob.sentences:
        polarity = sentence.sentiment.polarity
        if -1.0 <= polarity <= -0.5:
            return "Negative response"
        elif -0.5 < polarity <= 0:
            return "Slightly Negative response"
        elif 0 <= polarity < 0.5:
            return "Slightly Positive Response"
        else:
            return "Positive Response"

def match_articles(promise):
    cosine_sim = cosine_similarity(tfidf_matrix[promise: promise + 1], tfidf_matrix)
    single_array = np.array(cosine_sim[0])
    article_array = single_array.argsort()[-6:][::-1]
    only_articles = [s for s in article_array if s > 1]
    promise_statement = all_tokens[promise]
    print("The promise made by the candidate was: \n", promise_statement)
    count = 1
    for x in only_articles:
        print(count, all_tokens[x] + "\n")
        print(sentiment_analysis(all_tokens[x]))
        count += 1


# Political Promise Evaluation (PPE)
#### NLP CS6120
##### By: Bhanu Jain, Rohit Begani

# Inspirations

![](img/trumptracker.png)

![](img/politifact.png)

# Problem 
#### Track performance of political leaders throught continous evaluation of promises made during election campaign


# Data

## Promises
- Politifact 

## Promise Evaluation
- Twiiter 
- NYT
- Google Search

# Promises

In [57]:
with open("out/promises.json") as data_file:
    p_data = json.load(data_file)

len(p_data)

174

In [59]:
pprint.pprint(p_data[0])

{'promise_description': '',
 'promise_tag': ['Contract with the American Voter - 100 Day Plan'],
 'promise_title': 'Propose a Constitutional Amendment to impose term limits on '
                  'all members of Congress'}


In [55]:
promises = get_promise_token()
for k,v in promises.items():    
    print(k+":\n"+v+"\n")

Repeal and Replace Obamacare Act:
fully repeals obamacare and replaces it with health savings accounts the ability to purchase health insurance across state lines and lets states manage medicaid funds reforms will also include cutting the red tape at the fda there are over 4000 drugs awaiting approval and we especially want to speed the approval of lifesaving medications

Build a wall. Trump's campaign began with a promise to build a wall across the United States' southern border and deport the country's 11 million undocumented immigrants.:
build a wall trumps campaign began with a promise to build a wall across the united states southern border and deport the countrys 11 million undocumented immigrants



# Twitter

In [34]:
tweets = TwitterMiner().login().search.tweets(q="#trump", count=5)
for tweet in tweets['statuses']:
    print(tweet['text'])

Don't talk crap..

At least stand in respectful solidarity with the majority Syrian population &amp; their demands. You… https://t.co/7RJZtkqNqW
RT @RawStory: #RussiaRussiaRussia. #Trump has turned into #JanBrady. https://t.co/vDQQH3QPQ1
RT @ToDropADime2: If your Husband or Wife Lied to you 5 times a day, would you stay with them?

#Trump Lies 5.5 times a day......
RT @haaretzcom: Trump Tweets anti-Semitic, Conspiracy Theorist Website Boasting About His Accomplishments
https://t.co/grh6SN1PVx
#Trump #A…


# NYT

### Articles

In [29]:
nm = NewsMiner()
articles = nm.get_articles("Donald Trump", "20170730", "20170830")
nm.get_text(articles)
print(json.dumps(articles[0]['text'][:300],indent=4, sort_keys=True))

"In Damac, Mr. Trump\u2019s company has found a like-minded partner, one that has emblazoned its name across skyscrapers.\n\nAdvertisement Continue reading the main story\n\nThe company\u2019s founder, Hussain Sajwani, has a net worth estimated at around $4 billion, earning him the nickname \u201cthe Donald of Dubai,\u201d "


### Summary and Keywords

In [32]:
print(json.dumps(articles[0]['summary'],indent=4, sort_keys=True))

"He and his family spent New Year\u2019s Eve with Mr. Trump at his Florida resort, and he attended the president\u2019s inauguration.\nIn an interview with The Times last year, Mr. McLoughlin, the Damac spokesman, said Mr. Trump had visited Damac Hills several times.\nBuilding anything in Dubai invariably involves migrant workers, in transactions that often deviate from the law.\nIn response, the government has made it permissible for migrant workers to change jobs while outlawing payments to recruitment firms.\nAl Arif, the company supplying workers for the Trump course at Damac Hills, houses its workers in isolated apartment complexes in the desert."


In [33]:
print(articles[0]['keywords_1'])

['wages', 'reading', 'dubai', 'course', 'golf', 'trump', 'workers', 'main', 'donald', 'hills', 'damac', 'late', 'mr', 'migrant']


In [73]:
articles = get_article_token()
print("Total Articles Len = %s" %(len(articles)))

Total Articles Len = 130


In [77]:
print("Key: %s" %list(articles.keys())[0])
print("Text: %s" %articles[list(articles.keys())[0]][:300])

Key: 59934a3d95d0e0246f1ffdc7
Text: the exodus began monday when merck s chief executive said he was resigning from mr trump s manufacturing council citing the president s tepid early statement on the violence in charlottesville where white nationalists staged a weekend march that turned violent on tuesday the leaders of a labor group


# Methodology

# Article Sentiment 

## Article Matching 
- Bag of Words
- Tf-Idf

## Sentiment Analysis
- Naive Bayes
- Pattern 

# Google Summary Sentiment

# Aspect Based Senitment Analysis

# Experiments

# Whats Next